In [4]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import nodeleys as ndl
import sys
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl
import time
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)


In [5]:
ndl = t_update(ndl)
from nodeleys.model.layers import Dense
from nodeleys.model.layers.activations import *
from nodeleys.math.initializers import *
from nodeleys import Node

In [6]:
import time

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,516)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,516)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(516,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(516,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(gamma0, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_gamma0 + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
tr = []
loss.adic.begin_backprop(tracing=False, traces=tr)
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

140636143249808
140636143250192
--- 0.008705615997314453 seconds ---
--- 0.13380169868469238 seconds ---


In [7]:
w0.get_gradient()

array([[ 9.82822554e+07,  3.78434180e+07, -3.63123538e+07, ...,
        -4.96716203e+07,  4.14563141e+07, -3.11265443e+07],
       [ 5.71906523e+07,  2.03560911e+07, -3.16690264e+07, ...,
        -3.36158320e+07,  3.78552372e+07, -2.03696843e+07],
       [-6.38790652e+07, -3.27947696e+07,  3.98388023e+05, ...,
         2.69726888e+07, -1.22858760e+06,  1.31635847e+07],
       ...,
       [ 1.42492532e+08,  1.03406228e+08,  7.86138349e+07, ...,
        -5.05243591e+07, -9.86913610e+07, -1.16349858e+07],
       [ 1.11180528e+06,  2.96329495e+07,  8.58232666e+07, ...,
         1.65220999e+07, -1.05689392e+08,  2.08509678e+07],
       [ 8.85673440e+06, -9.53884602e+05, -2.29354333e+07, ...,
        -1.14135006e+07,  2.58732667e+07, -8.34205119e+06]])

In [8]:
grads[0]

<tf.Tensor: shape=(1516, 516), dtype=float64, numpy=
array([[ 9.82822554e+07,  3.78434180e+07, -3.63123538e+07, ...,
        -4.96716203e+07,  4.14563141e+07, -3.11265443e+07],
       [ 5.71906523e+07,  2.03560911e+07, -3.16690264e+07, ...,
        -3.36158320e+07,  3.78552372e+07, -2.03696843e+07],
       [-6.38790652e+07, -3.27947696e+07,  3.98388023e+05, ...,
         2.69726888e+07, -1.22858760e+06,  1.31635847e+07],
       ...,
       [ 1.42492532e+08,  1.03406228e+08,  7.86138349e+07, ...,
        -5.05243591e+07, -9.86913610e+07, -1.16349858e+07],
       [ 1.11180528e+06,  2.96329495e+07,  8.58232666e+07, ...,
         1.65220999e+07, -1.05689392e+08,  2.08509678e+07],
       [ 8.85673440e+06, -9.53884602e+05, -2.29354333e+07, ...,
        -1.14135006e+07,  2.58732667e+07, -8.34205119e+06]])>